In [6]:
# LightGBM and Bayesian Optimization Method 2
# code doesnt work :(
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from hyperopt import hp, tpe, fmin, STATUS_OK, Trials
from sklearn.metrics import classification_report, roc_auc_score

# Load the data
train_df = pd.read_csv('train_tfidf_features.csv')
test_df = pd.read_csv('test_tfidf_features.csv')
submission_df = pd.read_csv('sample_submission.csv')

# Extract features and labels
X = train_df.drop('label', axis=1)
y = train_df['label']

# Convert to sparse matrices 
X_sparse = csr_matrix(X.values)
test_sparse = csr_matrix(test_df.values)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_sparse, y, test_size=0.2, random_state=42)

# Define the objective function for hyperparameter tuning
def objective(params):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': int(params['num_leaves']),
        'max_depth': int(params['max_depth']),
        'learning_rate': params['learning_rate'],
        'feature_fraction': params['feature_fraction'],
        'bagging_fraction': params['bagging_fraction'],
        'bagging_freq': int(params['bagging_freq']),
        'min_child_samples': int(params['min_child_samples']),
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, eval_metric='auc', verbose=0)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, y_pred)
    
    return {'loss': -score, 'status': STATUS_OK}

# Define the hyperparameter space
space = {
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'feature_fraction': hp.uniform('feature_fraction', 0.6, 1.0),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.6, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
    'min_child_samples': hp.quniform('min_child_samples', 5, 30, 1)
}

# Optimize hyperparameters
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)
print(f"Best parameters: {best_params}")

# Convert hyperparameters to correct format
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['bagging_freq'] = int(best_params['bagging_freq'])
best_params['min_child_samples'] = int(best_params['min_child_samples'])

# Train the model with the best parameters using the LGBMClassifier
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, eval_metric='auc', verbose=0)

# Predict and evaluate on the validation set
y_pred_lgb = best_model.predict(X_val)
y_proba_lgb = best_model.predict_proba(X_val)[:, 1]

print("LightGBM Performance:")
print(classification_report(y_val, y_pred_lgb))
print(f"ROC AUC: {roc_auc_score(y_val, y_proba_lgb)}")

# Train on the full training data
best_model.fit(X_sparse, y)

# Predict on the test set
test_predictions = best_model.predict(test_sparse)

# Prepare submission file
submission_df['label'] = test_predictions
submission_df.to_csv('submission4.csv', index=False)


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

job exception: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'